In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [3]:
%%sql
SELECT DISTINCT(quantity) FROM invoice_line

Done.


quantity
1


In [4]:
%%sql
WITH usa_sales AS 
          (SELECT il.* FROM invoice_line il
           INNER JOIN invoice i on il.invoice_id = i.invoice_id
           INNER JOIN customer c on i.customer_id = c.customer_id
           WHERE c.country = "USA")
    
SELECT g.name genre_name,
       SUM(us.quantity) amount_by_genre,
       ROUND(CAST(SUM(us.quantity) AS FLOAT)/ (SELECT SUM(quantity) FROM usa_sales), 3) pct_by_genre
FROM usa_sales us
INNER JOIN track t ON t.track_id = us.track_id
INNER JOIN genre g ON g.genre_id = t.genre_id
GROUP BY 1
ORDER BY 2 DESC

Done.


genre_name,amount_by_genre,pct_by_genre
Rock,561,0.534
Alternative & Punk,130,0.124
Metal,124,0.118
R&B/Soul,53,0.05
Blues,36,0.034
Alternative,35,0.033
Latin,22,0.021
Pop,22,0.021
Hip Hop/Rap,20,0.019
Jazz,14,0.013


Based on the popularity of music genres among American consumers, we recommend:
* *Red Tone*  - Punk
* *Meteor and the Girls* - Pop
* *Slim Jim Bites* -	Blues

In [5]:
%%sql
WITH customer_spend AS 
(SELECT c.customer_id,
        c.support_rep_id,
        SUM(i.total) total_spent
FROM customer c
INNER JOIN invoice i ON i.customer_id = c.customer_id
GROUP BY 1)

SELECT e.employee_id,
       e.first_name || ' ' || e.last_name name,
       e.title,
       e.hire_date,
       COUNT(cs.customer_id) customer_num,
       ROUND(SUM(cs.total_spent), 2) customer_spent
FROM employee e 
INNER JOIN customer_spend cs ON cs.support_rep_id = e.employee_id
GROUP BY 1

Done.


employee_id,name,title,hire_date,customer_num,customer_spent
3,Jane Peacock,Sales Support Agent,2017-04-01 00:00:00,21,1731.51
4,Margaret Park,Sales Support Agent,2017-05-03 00:00:00,20,1584.0
5,Steve Johnson,Sales Support Agent,2017-10-17 00:00:00,18,1393.92


Based on the total number and expense of customers each sales support agent supports, with their hire date, looks like the amount of work is evenly distributed. All three of the sales support are performing similarly. 

In [6]:
%%sql
WITH customer_by_country AS 
     (SELECT 
       (CASE 
       WHEN COUNT(DISTINCT c.customer_id) = 1 THEN 'Other'
       ELSE c.country 
       END) AS country,
       COUNT(DISTINCT c.customer_id) customer_num,
       SUM(i.total) total,
       COUNT(i.invoice_id) order_quantity
    FROM customer c
    INNER JOIN invoice i ON i.customer_id = c.customer_id
    GROUP BY country)

SELECT country,
       SUM(customer_num) customer_num,
       ROUND(SUM(total), 2) total_sales,
       ROUND(SUM(total)/ SUM(customer_num), 2) avg_sales_per_customer, 
       ROUND(SUM(total)/ SUM(order_quantity), 2) avg_order_value
FROM (SELECT ctc.*,
             CASE
          WHEN country = 'Other' THEN 1
          ELSE 0 
          END AS sort 
      FROM customer_by_country ctc)
GROUP BY 1
ORDER BY sort, 2     

Done.


country,customer_num,total_sales,avg_sales_per_customer,avg_order_value
Czech Republic,2,273.24,136.62,9.11
India,2,183.15,91.57,8.72
Portugal,2,185.13,92.57,6.38
United Kingdom,3,245.52,81.84,8.77
Germany,4,334.62,83.66,8.16
Brazil,5,427.68,85.54,7.01
France,5,389.07,77.81,7.78
Canada,8,535.59,66.95,7.05
USA,13,1040.49,80.04,7.94
Other,15,1094.94,73.0,7.45


In [77]:
%%sql
WITH 
track_album AS 
(SELECT t.album_id,
        COUNT(t.track_id) track_num      
FROM track t
INNER JOIN album a ON a.album_id = t.album_id
GROUP BY 1),

orders AS 
(SELECT il.invoice_id,
        a.album_id,
        COUNT(il.track_id) track_num
FROM invoice_line il
INNER JOIN track t ON t.track_id = il.track_id
INNER JOIN album a ON a.album_id = t.album_id
GROUP BY 1,2),

purchase_cat AS 
(SELECT CASE 
       WHEN o.track_num = ta.track_num THEN 'album' 
       ELSE 'single'
       END AS purchase_cat,
       o.invoice_id
FROM orders o
INNER JOIN track_album ta ON ta.album_id = o.album_id),

edge_free_cat AS 
(SELECT CASE
       WHEN COUNT(DISTINCT(purchase_cat))= 2 THEN 'single'
       ELSE purchase_cat
       END AS purchase_cat,
       invoice_id
FROM purchase_cat
GROUP BY invoice_id)

SELECT purchase_cat,
       COUNT(invoice_id) amount,
       ROUND(CAST(COUNT(invoice_id) AS FLOAT)/ (SELECT COUNT(invoice_id) FROM edge_free_cat),2) percentage
FROM edge_free_cat
GROUP BY purchase_cat

Done.


purchase_cat,amount,percentage
album,114,0.19
single,500,0.81
